In [ ]:
import pycinema
import ipywidgets
import IPython

# pycinema.Filter._debug = True

# Layout
parameterWidgetsContainer = ipywidgets.VBox();
colorMappingWidgetsContainer = [ipywidgets.VBox(),ipywidgets.VBox()];
shadingWidgetsContainer = [ipywidgets.VBox(),ipywidgets.VBox()];

leftColumn = ipywidgets.VBox([
    ipywidgets.Accordion(children=[parameterWidgetsContainer]),
    ipywidgets.Accordion(children=[colorMappingWidgetsContainer[0]]),
    ipywidgets.Accordion(children=[colorMappingWidgetsContainer[1]]),
    ipywidgets.Accordion(children=[shadingWidgetsContainer[0]]),
    ipywidgets.Accordion(children=[shadingWidgetsContainer[1]])
]);
leftColumn.layout.min_width = '28em'
leftColumn.layout.max_width = '28em'
leftColumn.children[0].set_title(0,'Parameters')
leftColumn.children[1].set_title(0,'Color Mapping A')
leftColumn.children[2].set_title(0,'Color Mapping B')
leftColumn.children[3].set_title(0,'Shading A')
leftColumn.children[4].set_title(0,'Shading B')

imageContainer = [ipywidgets.HBox(),ipywidgets.HBox()]
globalContainer = ipywidgets.HBox([leftColumn,ipywidgets.HBox(imageContainer)]);

# Pipeline
cinemaDatabaseReader = [pycinema.CinemaDatabaseReader(),pycinema.CinemaDatabaseReader()]

databaseMerger  = pycinema.DatabaseMerger()
databaseMerger.inputs.table0.set(cinemaDatabaseReader[0].outputs.table,False)
databaseMerger.inputs.table1.set(cinemaDatabaseReader[1].outputs.table,False)

parameterWidgets = pycinema.ParameterWidgets()
parameterWidgets.inputs.table.set(databaseMerger.outputs.table,False)
parameterWidgets.inputs.container.set(parameterWidgetsContainer,False)

for i in range(0,2):
    databaseQuery = pycinema.DatabaseQuery()
    databaseQuery.inputs.table.set(cinemaDatabaseReader[i].outputs.table,False)
    databaseQuery.inputs.sql.set(parameterWidgets.outputs.sql,False)

    imageReader = pycinema.ImageReader()
    imageReader.inputs.table.set(databaseQuery.outputs.table,False)

    depthCompositing = pycinema.DepthCompositing()
    depthCompositing.inputs.images_a.set(imageReader.outputs.images,False)
    depthCompositing.inputs.composite_by_meta.set(parameterWidgets.outputs.composite_by_meta,False)

    colorMappingWidgets = pycinema.ColorMappingWidgets()
    colorMappingWidgets.inputs.images.set(depthCompositing.outputs.images,False)
    colorMappingWidgets.inputs.container.set(colorMappingWidgetsContainer[i],False)

    colorMapping = pycinema.ColorMapping()
    colorMapping.inputs.images.set(depthCompositing.outputs.images,False)
    colorMapping.inputs.map.set(colorMappingWidgets.outputs.map,False)
    colorMapping.inputs.range.set(colorMappingWidgets.outputs.range,False)
    colorMapping.inputs.channel.set(colorMappingWidgets.outputs.channel,False)
    colorMapping.inputs.nan.set(colorMappingWidgets.outputs.nan,False)

    shaderIBS = pycinema.ShaderIBS()
    shaderIBS.inputs.images.set(colorMapping.outputs.images,False)

    shadingWidgetsContainer[i].children = [
        pycinema.NumberWidget(shaderIBS.inputs.radius, range=[0,1,0.01]).widget,
        pycinema.NumberWidget(shaderIBS.inputs.samples, range=[1,256,1]).widget,
        pycinema.NumberWidget(shaderIBS.inputs.silhouette, range=[0.01,1,0.01]).widget,
        pycinema.NumberWidget(shaderIBS.inputs.luminance, range=[0,2,0.01]).widget,
        pycinema.NumberWidget(shaderIBS.inputs.ambient, range=[0,2,0.01]).widget,
    ]

    shaderFXAA = pycinema.ShaderFXAA()
    shaderFXAA.inputs.images.set(shaderIBS.outputs.images,False)

    annotation = pycinema.Annotation()
    annotation.inputs.images.set(shaderFXAA.outputs.images,False)

    imageViewer = pycinema.ImageViewer()
    imageViewer.inputs.images.set( annotation.outputs.images, False )
    imageViewer.inputs.container.set(imageContainer[i],False)

IPython.display.display(globalContainer)

cinemaDatabaseReader[0].inputs.path.set('../../pycinema-data/ViscousFingers_Geometry.cdb',False)
cinemaDatabaseReader[1].inputs.path.set('../../pycinema-data/ViscousFingers_PersistenceDiagram.cdb')